<a href="https://colab.research.google.com/github/Abdulrahman-Ayman-Farid/ChatBot-Project/blob/main/Nasa_Farming_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rarfile
import zipfile
import os
import rarfile

# Define the path to the uploaded zip file (replace 'new_zip_file.zip' with your actual file name)
zip_path = '/content/Nasa Frames.rar'

# Extract the RAR file
with rarfile.RarFile(zip_path, 'r') as rar_ref:
    rar_ref.extractall('/content/frames1')  # Extracts to a folder named 'frames'

# Check if extraction was successful
print(f"Extracted files: {os.listdir('/content/frames1')}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import OpenEXR
import numpy as np
import os

# Enable OpenEXR support in OpenCV
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"

def load_exr_frame(filepath):
    # Read the .exr file using OpenCV
    try:
        exr_image = cv2.imread(filepath, cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

    # Check if the image was loaded properly
    if exr_image is None:
        print(f"Could not load image {filepath}") # Changed from raise ValueError to print
        return None # Return None instead of raising an error

    # Normalize the data if necessary
    exr_image = cv2.normalize(exr_image, None, 0, 1, cv2.NORM_MINMAX)

    return exr_image

# List all .exr files in the extracted folder
frames_folder = '/content/frames1'  # Adjust the path as needed
exr_files = [os.path.join(frames_folder, f) for f in os.listdir(frames_folder) if f.endswith('.exr')]

# Load all frames into a list
frames_list = []
for filepath in exr_files:
    frame = load_exr_frame(filepath)
    if frame is not None: # Only add the frame if it was loaded successfully
        frames_list.append(frame)

# Convert the list to a NumPy array for model training
frames_array = np.array(frames_list)

# Check the shape of the loaded frames array
print(f"Frames shape: {frames_array.shape}")

In [ ]:
!file /content/GMAO_extremes_map_2023-03-01_2057.00001.exr


/content/frames1/GMAO_extremes_map_2023-03-01_2057.00004.exr: HTML document, ASCII text


In [ ]:
import OpenEXR
import numpy as np
import Imath

def load_exr_frame(filepath):
    try:
        exr_file = OpenEXR.InputFile(filepath)
        header = exr_file.header()
        channels = header['channels'].keys()

        # Read the R, G, B channels
        rgb = [exr_file.channel(c, Imath.PixelType(Imath.PixelType.FLOAT)) for c in channels]

        # Get dataWindow dimensions
        dw = header['dataWindow']
        width = dw.max.x - dw.min.x + 1
        height = dw.max.y - dw.min.y + 1
        shape = (height, width)

        # Reshape the channel data to the image dimensions
        rgb = [np.frombuffer(c, dtype=np.float32).reshape(shape) for c in rgb]
        exr_image = np.stack(rgb, axis=-1)

        # Normalize the image (if needed)
        exr_image = (exr_image - np.min(exr_image)) / (np.max(exr_image) - np.min(exr_image))

        return exr_image

    except Exception as e:
        print(f"Error loading image {filepath}: {e}")
        return None

# Test the function
frame = load_exr_frame('/content/frames1/GMAO_extremes_map_2023-03-01_2057.00001.exr')
if frame is not None:
    print(f"Frame loaded successfully with shape {frame.shape}")


Error loading image /content/frames1/GMAO_extremes_map_2023-03-01_2057.00001.exr: Cannot read image file "/content/frames1/GMAO_extremes_map_2023-03-01_2057.00001.exr". File is not an image file.
